In [ ]:
!pip install prophet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import requests
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')  # Hide warnings

from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from platform import python_version
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_yearly
from scipy.stats import norm

# Know the setup

In [ ]:
print(python_version())

## Define var for furture usage

In [ ]:
code = 'INTC'

# Load the dataset

In [ ]:
df = pd.read_csv('../input/sp500-spx-components-stock-price/data/' + code + '.csv')

# Data Understanding

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.info()

# Data Preparation

# Checking invalid value

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

# Drop invalid value

In [ ]:
df.dropna()

# Data Transform

## Convert Date field from object datatype into datetime datatype

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

## Select sepecific time range

### Select 20 years data

In [ ]:
df = df[df['Date'].between(datetime.now() - relativedelta(years=int(20)), datetime.now())]

In [ ]:
df = df[["Date","Adj Close"]]
data = df.rename(columns={"Date": "ds", "Adj Close": "y"})

# Getting Extra Quarterly Revenue Data with BratifulSoup

In [ ]:
url = 'https://www.macrotrends.net/stocks/charts/' + code + '/details/revenue'
html_text = requests.get(url).text
soup=BeautifulSoup(html_text, 'html5lib')
quarterly_revenue = []

tables = soup.find_all('table')
table_index = 0
for index, table in enumerate(tables):
    if ('Quarterly Revenue'in str(table)):
        table_index=index
for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        date = col[0].text
        quarterly_revenue.append(date)
quarterly_revenue

In [ ]:
quarterly_revenue = pd.DataFrame({
  'holiday': 'Quarterly Revenue',
  'ds': pd.to_datetime(quarterly_revenue),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = quarterly_revenue

# Data Modelling

In [ ]:
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True, holidays=holidays)
model.add_country_holidays(country_name='US')
model.fit(data)

In [ ]:
future = model.make_future_dataframe(periods=365)
predict = model.predict(future)

In [ ]:
model.train_holiday_names

# Data Visualization

In [ ]:
fig_model_plot = model.plot(predict)

In [ ]:
fig_model_plot_components = model.plot_components(predict)

In [ ]:
fig_model_plot_yearly = plot_yearly(model)

In [ ]:
plot_plotly(model, predict)

In [ ]:
plot_components_plotly(model, predict)

# Extra: Log transform

In [ ]:
data.tail()

In [ ]:
data_log_transform = data

In [ ]:
data_log_transform['y'] = np.log(data_log_transform['y'])

In [ ]:
data_log_transform.tail()

In [ ]:
model_log_transform = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True, holidays=holidays)
model_log_transform.add_country_holidays(country_name='US')
model_log_transform.fit(data_log_transform)

In [ ]:
future_log_transform = model_log_transform.make_future_dataframe(periods=365)
predict_log_transform = model_log_transform.predict(future_log_transform)

In [ ]:
fig_model_log_transform_plot = model_log_transform.plot(predict_log_transform)

In [ ]:
fig_model_log_transform_plot_components = model_log_transform.plot_components(predict_log_transform)

In [ ]:
fig_model_log_transform_plot_yearly = plot_yearly(model_log_transform)

In [ ]:
plot_plotly(model_log_transform, predict_log_transform)

In [ ]:
plot_components_plotly(model_log_transform, predict_log_transform)

Thanks you for your watching. Please let me know your though if any.

Hope you have a great day!